# Cloud Advanced Downloader

Multi-engine download manager for Google Colab. Supports Aria2, Wget, Curl, and Mega.nz with parallel or sequential download modes.

---
**Run each cell in order. All 4 cells must be executed.**

In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 1 — Mount Google Drive (Optional)                     ║
# ╚══════════════════════════════════════════════════════════════╝

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time, os

# ── Banner ────────────────────────────────────────────────────────────────────
display(HTML("""
<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%);
    border-radius: 12px; padding: 28px 32px; margin: 8px 0;
    font-family: 'Segoe UI', sans-serif;
    border: 1px solid rgba(255,255,255,0.08);
    box-shadow: 0 8px 32px rgba(0,0,0,0.4);
    text-align: center;
">
    <div style='font-size:1.8em; font-weight:700; color:#e2e8f0; letter-spacing:3px;'>
        CLOUD ADVANCED DOWNLOADER
    </div>
    <div style='color:#475569; font-size:0.8em; letter-spacing:5px; margin-top:6px;'>
        ARIA2  ·  WGET  ·  CURL  ·  MEGA
    </div>
</div>
"""))

display(HTML("""
<div style="background:#0f172a; border-radius:8px; padding:14px 18px; margin:8px 0;
     font-family:monospace; font-size:0.85em; color:#94a3b8; border:1px solid #1e293b;">
    <span style='color:#3b82f6; font-weight:bold;'>Default download folder:</span>
    <span style='color:#f0abfc;'> /content/download</span><br>
    <span style='color:#64748b;'>Toggle below to also mount Google Drive for direct transfer.</span>
</div>
"""))

# ── Mount Toggle ──────────────────────────────────────────────────────────────
mount_toggle = widgets.ToggleButton(
    value=False,
    description='Mount Google Drive',
    button_style='',
    icon='cloud',
    style={'button_color': '#1e293b', 'font_weight': 'bold'},
    layout=widgets.Layout(width='220px', height='40px')
)

mount_btn = widgets.Button(
    description='Apply',
    button_style='primary',
    layout=widgets.Layout(width='100px', height='40px')
)

mount_out = widgets.Output()

def on_mount_click(b):
    with mount_out:
        clear_output(wait=True)
        if mount_toggle.value:
            display(HTML("""
            <div style='background:#0f172a; border-radius:8px; padding:14px 18px;
                 border:1px solid #1e293b; font-family:monospace; color:#94a3b8;'>
                <div style='color:#e2e8f0; margin-bottom:10px;'>Mounting Google Drive...</div>
                <div style='background:#1e293b; border-radius:4px; height:6px; overflow:hidden;'>
                    <div style='height:100%; background:linear-gradient(90deg,#3b82f6,#8b5cf6);
                         animation:bar 1.5s ease-in-out infinite; width:40%; margin-left:0%;'></div>
                </div>
            </div>
            <style>@keyframes bar{0%{margin-left:0%;width:20%}50%{margin-left:40%;width:40%}100%{margin-left:80%;width:20%}}</style>
            """))
            try:
                from google.colab import drive
                drive.mount('/content/drive')
                clear_output(wait=True)
                display(HTML("""
                <div style='background:#14532d; border-left:4px solid #22c55e;
                     border-radius:6px; padding:12px 16px; font-family:sans-serif; color:#bbf7d0;'>
                    <strong>Google Drive mounted successfully.</strong><br>
                    <span style='font-size:0.85em;'>Path: /content/drive/MyDrive</span>
                </div>
                """))
            except Exception as e:
                clear_output(wait=True)
                display(HTML(f"""
                <div style='background:#7f1d1d; border-left:4px solid #ef4444;
                     border-radius:6px; padding:12px 16px; font-family:sans-serif; color:#fecaca;'>
                    <strong>Mount failed:</strong> {e}
                </div>
                """))
        else:
            display(HTML("""
            <div style='background:#1e293b; border-left:4px solid #64748b;
                 border-radius:6px; padding:12px 16px; font-family:sans-serif; color:#94a3b8;'>
                Google Drive skipped. Files will be saved to <code>/content/download</code>.
            </div>
            """))

mount_btn.on_click(on_mount_click)
display(widgets.HBox([mount_toggle, mount_btn]), mount_out)
os.makedirs('/content/download', exist_ok=True)
print()


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 2 — Installation & Setup                              ║
# ╚══════════════════════════════════════════════════════════════╝

import subprocess, sys, time
from IPython.display import display, HTML, clear_output

def run_cmd(cmd):
    return subprocess.run(cmd, shell=True, capture_output=True, text=True)

def show_install_step(step_name, pct, total_steps=6):
    filled = int(pct / 5)
    bar = '█' * filled + '░' * (20 - filled)
    display(HTML(f"""
    <div style="background:#0f172a; border-radius:8px; padding:14px 18px; margin:4px 0;
         font-family:monospace; color:#94a3b8; border:1px solid #1e293b;">
        <div style="color:#e2e8f0; margin-bottom:8px; font-size:0.9em;">
            Installing: <span style='color:#f0abfc;'>{step_name}</span>
        </div>
        <div style="display:flex; align-items:center; gap:10px;">
            <span style='color:#3b82f6;'>[{bar}]</span>
            <span style='color:#22c55e; font-weight:bold;'>{pct}%</span>
        </div>
    </div>
    """))

clear_output(wait=True)
display(HTML("""
<div style="background:#0f172a; border-radius:10px; padding:16px 20px; margin:8px 0;
     border:1px solid #1e293b; font-family:monospace; color:#64748b;
     font-size:0.8em; letter-spacing:2px; text-transform:uppercase;">
    INSTALLATION & ENGINE SETUP
</div>
"""))

steps = [
    ("System packages",       "apt-get update -qq && apt-get install -y -qq aria2 megatools wget curl 2>/dev/null || apt-get install -y -qq aria2 wget curl"),
    ("Python: pip upgrade",   f"{sys.executable} -m pip install --upgrade pip -q"),
    ("Python: tqdm",          f"{sys.executable} -m pip install tqdm -q"),
    ("Python: requests",      f"{sys.executable} -m pip install requests -q"),
    ("Python: mega.py",       f"{sys.executable} -m pip install mega.py -q"),
    ("Aria2 configuration",   "echo '[OK] Configuring aria2...'"),
]

total = len(steps)
install_log = []

for i, (name, cmd) in enumerate(steps):
    pct = int(((i) / total) * 100)
    clear_output(wait=True)
    display(HTML("""
    <div style="background:#0f172a; border-radius:10px; padding:16px 20px; margin:8px 0;
         border:1px solid #1e293b; font-family:monospace; color:#64748b;
         font-size:0.8em; letter-spacing:2px;">
        INSTALLATION & ENGINE SETUP
    </div>
    """))
    for j, (pname, _) in enumerate(steps[:i]):
        display(HTML(f"<div style='font-family:monospace; font-size:0.82em; color:#22c55e; margin:2px 0;'>  [OK] {pname}</div>"))
    show_install_step(name, pct)
    result = run_cmd(cmd)
    ok = result.returncode == 0
    install_log.append((name, ok, result.stderr[:100] if not ok else ''))

# Configure aria2
os.makedirs('/root/.aria2', exist_ok=True)
aria2_conf = """
continue=true
max-connection-per-server=16
split=16
min-split-size=1M
max-concurrent-downloads=5
allow-overwrite=true
auto-file-renaming=false
console-log-level=warn
"""
with open('/root/.aria2/aria2.conf', 'w') as f:
    f.write(aria2_conf)

clear_output(wait=True)
display(HTML("""
<div style="background:#0f172a; border-radius:10px; padding:16px 20px; margin:8px 0;
     border:1px solid #1e293b; font-family:monospace; color:#64748b;
     font-size:0.8em; letter-spacing:2px;">INSTALLATION & ENGINE SETUP</div>
"""))
for name, ok, err in install_log:
    color = '#22c55e' if ok else '#f87171'
    icon  = '[OK]' if ok else '[!!]'
    extra = f" — {err}" if err else ''
    display(HTML(f"<div style='font-family:monospace; font-size:0.82em; color:{color}; margin:2px 0;'>  {icon} {name}{extra}</div>"))

display(HTML("""
<div style='background:#14532d; border-left:4px solid #22c55e; border-radius:6px;
     padding:12px 16px; margin-top:12px; font-family:sans-serif; color:#bbf7d0;'>
    <strong>Setup complete.</strong> All engines are ready. Proceed to Cell 3.
</div>
"""))


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 3 — Download Manager                                  ║
# ╚══════════════════════════════════════════════════════════════╝

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import subprocess, threading, os, time

DOWNLOAD_DIR = '/content/download'
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# ── Engine descriptions ───────────────────────────────────────────────────────
ENGINE_INFO = {
    'Aria2':      {'pros': 'Fastest. Multi-connection parallel download. Resumable.', 'cons': 'Some servers block multi-connection.'},
    'Wget':       {'pros': 'Universal. Recursive support. Resumable.', 'cons': 'Single connection. Slower.'},
    'Curl':       {'pros': 'Handles redirects & cookies well.', 'cons': 'Single connection.'},
    'Mega Auto':  {'pros': 'Bypasses quota limits. Resume support.', 'cons': 'Mega.nz links only.'},
}

# ── Widgets ───────────────────────────────────────────────────────────────────
style = {'description_width': '160px'}
layout_full  = widgets.Layout(width='100%')
layout_half  = widgets.Layout(width='50%')
layout_input = widgets.Layout(width='100%', height='36px')

engine_dd = widgets.Dropdown(
    options=['Aria2', 'Wget', 'Curl', 'Mega Auto'],
    value='Aria2',
    description='Engine:',
    style=style, layout=widgets.Layout(width='280px')
)

mode_dd = widgets.Dropdown(
    options=['Sequential (one by one)', 'Parallel (all at once)'],
    value='Sequential (one by one)',
    description='Mode:',
    style=style, layout=widgets.Layout(width='300px')
)

url_inputs = [
    widgets.Text(placeholder=f'Link {i+1}', layout=layout_input)
    for i in range(5)
]

conn_slider = widgets.IntSlider(
    value=16, min=1, max=32, step=1,
    description='Aria2 Connections:',
    style=style, layout=layout_full,
    continuous_update=False
)

split_slider = widgets.IntSlider(
    value=16, min=1, max=32, step=1,
    description='Aria2 Splits:',
    style=style, layout=layout_full,
    continuous_update=False
)

download_btn = widgets.Button(
    description='Start Download',
    button_style='success',
    icon='download',
    layout=widgets.Layout(width='180px', height='44px')
)

clear_btn = widgets.Button(
    description='Clear All',
    button_style='warning',
    layout=widgets.Layout(width='120px', height='44px')
)

output_area = widgets.Output()
engine_info_out = widgets.Output()

# ── Engine info display ───────────────────────────────────────────────────────
def update_engine_info(change=None):
    eng = engine_dd.value
    info = ENGINE_INFO[eng]
    aria2_visible = eng == 'Aria2'
    conn_slider.layout.display = '' if aria2_visible else 'none'
    split_slider.layout.display = '' if aria2_visible else 'none'
    with engine_info_out:
        clear_output(wait=True)
        display(HTML(f"""
        <div style="background:#0f172a; border-radius:8px; padding:12px 16px; margin:4px 0;
             font-family:sans-serif; font-size:0.85em; border:1px solid #1e293b;">
            <span style='color:#3b82f6; font-weight:bold;'>{eng}</span><br>
            <span style='color:#22c55e;'>+ {info['pros']}</span><br>
            <span style='color:#f87171;'>- {info['cons']}</span>
        </div>
        """))

engine_dd.observe(update_engine_info, names='value')

# ── Download logic ────────────────────────────────────────────────────────────
def build_aria2_cmd(url, connections, splits):
    return ['aria2c', f'--max-connection-per-server={connections}',
            f'--split={splits}', '--min-split-size=1M', '--continue=true',
            '--allow-overwrite=true', '--auto-file-renaming=false',
            f'--dir={DOWNLOAD_DIR}', url]

def build_wget_cmd(url):
    return ['wget', '-c', '-P', DOWNLOAD_DIR, url]

def build_curl_cmd(url):
    filename = url.split('/')[-1].split('?')[0] or 'download'
    return ['curl', '-L', '-C', '-', '-o', f'{DOWNLOAD_DIR}/{filename}', url]

def build_mega_cmd(url):
    return ['bash', '-c',
        f'megadl --path {DOWNLOAD_DIR} "{url}" 2>/dev/null || '
        f'python3 -c "from mega import Mega; m=Mega().login(); m.download_url(\'{url}\', dest_path=\'{DOWNLOAD_DIR}\')"'
    ]

def status_html(icon, color, title, msg):
    bg = {'green': '#14532d', 'red': '#7f1d1d', 'blue': '#1e3a5f', 'yellow': '#713f12'}[color]
    ac = {'green': '#22c55e', 'red': '#ef4444', 'blue': '#3b82f6', 'yellow': '#f59e0b'}[color]
    tc = {'green': '#bbf7d0', 'red': '#fecaca', 'blue': '#bfdbfe', 'yellow': '#fde68a'}[color]
    return f"""
    <div style='background:{bg}; border-left:4px solid {ac}; border-radius:6px;
         padding:10px 14px; margin:4px 0; font-family:sans-serif; color:{tc};'>
        <strong>{icon} {title}</strong><br>
        <span style='font-size:0.85em; opacity:0.85;'>{msg}</span>
    </div>
    """

def run_download(url, engine, connections, splits):
    if engine == 'Aria2':
        cmd = build_aria2_cmd(url, connections, splits)
    elif engine == 'Wget':
        cmd = build_wget_cmd(url)
    elif engine == 'Curl':
        cmd = build_curl_cmd(url)
    else:
        cmd = build_mega_cmd(url)
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.returncode == 0, result.stderr

def on_download(b):
    with output_area:
        clear_output(wait=True)
        urls = [u.value.strip() for u in url_inputs if u.value.strip()]
        if not urls:
            display(HTML(status_html('!', 'yellow', 'No URLs', 'Please enter at least one link.')))
            return

        engine = engine_dd.value
        parallel = 'Parallel' in mode_dd.value
        connections = conn_slider.value
        splits = split_slider.value

        display(HTML(f"""
        <div style='background:#0f172a; border-radius:8px; padding:12px 16px; margin:4px 0;
             font-family:monospace; font-size:0.85em; color:#94a3b8; border:1px solid #1e293b;'>
            Engine: <span style='color:#f0abfc;'>{engine}</span>  |
            Mode: <span style='color:#f0abfc;'>{'Parallel' if parallel else 'Sequential'}</span>  |
            Files: <span style='color:#f0abfc;'>{len(urls)}</span>
        </div>
        """))

        if parallel:
            results = {}
            threads = []

            def worker(url):
                ok, err = run_download(url, engine, connections, splits)
                results[url] = (ok, err)

            display(HTML(status_html('>', 'blue', 'Parallel Download', f'Launching {len(urls)} simultaneous download(s)...')))

            for url in urls:
                t = threading.Thread(target=worker, args=(url,))
                threads.append(t)
                t.start()

            for t in threads:
                t.join()

            for url, (ok, err) in results.items():
                short = url[:55] + '...' if len(url) > 55 else url
                if ok:
                    display(HTML(status_html('OK', 'green', short, 'Download complete.')))
                else:
                    display(HTML(status_html('!!', 'red', short, err[:120])))
        else:
            for i, url in enumerate(urls):
                short = url[:55] + '...' if len(url) > 55 else url
                display(HTML(status_html('>', 'blue', f'[{i+1}/{len(urls)}] Downloading', short)))
                ok, err = run_download(url, engine, connections, splits)
                if ok:
                    display(HTML(status_html('OK', 'green', f'File {i+1} complete', short)))
                else:
                    display(HTML(status_html('!!', 'red', f'File {i+1} failed', err[:120])))

        # Summary
        files = [f for f in os.listdir(DOWNLOAD_DIR) if os.path.isfile(f'{DOWNLOAD_DIR}/{f}')]
        display(HTML(f"""
        <div style='background:#14532d; border-left:4px solid #22c55e; border-radius:6px;
             padding:12px 16px; margin-top:8px; font-family:sans-serif; color:#bbf7d0;'>
            <strong>Done.</strong> {len(files)} file(s) in <code>/content/download</code>.<br>
            <span style='font-size:0.85em; opacity:0.75;'>Run Cell 4 to transfer to Google Drive.</span>
        </div>
        """))

def on_clear(b):
    for u in url_inputs:
        u.value = ''
    with output_area:
        clear_output()

download_btn.on_click(on_download)
clear_btn.on_click(on_clear)

# ── Layout ────────────────────────────────────────────────────────────────────
display(HTML('<div style="font-family:monospace; font-size:0.75em; color:#475569; letter-spacing:2px; margin-bottom:8px;">DOWNLOAD MANAGER</div>'))

display(widgets.HBox([engine_dd, mode_dd]))
display(engine_info_out)
update_engine_info()

display(HTML('<div style="font-family:monospace; font-size:0.75em; color:#475569; margin:10px 0 4px;">LINKS (up to 5)</div>'))
for u in url_inputs:
    display(u)

display(HTML('<div style="font-family:monospace; font-size:0.75em; color:#475569; margin:10px 0 4px;">ARIA2 SETTINGS</div>'))
display(conn_slider)
display(split_slider)

display(widgets.HBox([download_btn, clear_btn], layout=widgets.Layout(margin='12px 0')))
display(output_area)


In [ ]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  CELL 4 — Transfer to Google Drive                          ║
# ╚══════════════════════════════════════════════════════════════╝

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import subprocess, os, shutil

SRC_DIR = '/content/download'

drive_path = widgets.Text(
    value='Downloads',
    placeholder='Folder name in MyDrive (e.g. Downloads)',
    description='Drive Folder:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='420px')
)

transfer_btn = widgets.Button(
    description='Transfer to Drive',
    button_style='primary',
    icon='upload',
    layout=widgets.Layout(width='180px', height='44px')
)

refresh_btn = widgets.Button(
    description='List Files',
    button_style='',
    layout=widgets.Layout(width='120px', height='44px')
)

transfer_out = widgets.Output()

def format_size(n):
    for u in ['B','KB','MB','GB']:
        if n < 1024: return f'{n:.1f} {u}'
        n /= 1024
    return f'{n:.1f} TB'

def list_files_html():
    if not os.path.exists(SRC_DIR):
        return "<em style='color:#64748b;'>No download folder found.</em>"
    files = [f for f in os.listdir(SRC_DIR) if os.path.isfile(f'{SRC_DIR}/{f}')]
    if not files:
        return "<em style='color:#64748b;'>No files in /content/download yet.</em>"
    rows = ''.join(
        f"<tr><td style='padding:6px 10px; color:#e2e8f0;'>{f}</td>"
        f"<td style='padding:6px 10px; color:#94a3b8; text-align:right;'>"
        f"{format_size(os.path.getsize(f'{SRC_DIR}/{f}'))}</td></tr>"
        for f in files
    )
    return f"""
    <table style='width:100%; border-collapse:collapse; background:#0f172a;
           border-radius:8px; overflow:hidden; font-family:monospace; font-size:0.85em;'>
        <thead><tr>
            <th style='padding:8px 10px; color:#64748b; text-align:left; border-bottom:1px solid #1e293b;'>Filename</th>
            <th style='padding:8px 10px; color:#64748b; text-align:right; border-bottom:1px solid #1e293b;'>Size</th>
        </tr></thead>
        <tbody>{rows}</tbody>
    </table>
    """

def on_refresh(b):
    with transfer_out:
        clear_output(wait=True)
        display(HTML(list_files_html()))

def on_transfer(b):
    with transfer_out:
        clear_output(wait=True)

        drive_base = '/content/drive/MyDrive'
        if not os.path.exists(drive_base):
            display(HTML("""
            <div style='background:#7f1d1d; border-left:4px solid #ef4444; border-radius:6px;
                 padding:12px 16px; font-family:sans-serif; color:#fecaca;'>
                <strong>Google Drive not mounted.</strong> Please run Cell 1 and mount Drive first.
            </div>
            """))
            return

        folder_name = drive_path.value.strip() or 'Downloads'
        dest = os.path.join(drive_base, folder_name)
        os.makedirs(dest, exist_ok=True)

        files = [f for f in os.listdir(SRC_DIR) if os.path.isfile(f'{SRC_DIR}/{f}')]
        if not files:
            display(HTML("""
            <div style='background:#713f12; border-left:4px solid #f59e0b; border-radius:6px;
                 padding:12px 16px; font-family:sans-serif; color:#fde68a;'>
                <strong>No files to transfer.</strong> Download something first in Cell 3.
            </div>
            """))
            return

        display(HTML(f"""
        <div style='background:#0f172a; border-radius:8px; padding:12px 16px; margin:4px 0;
             font-family:monospace; font-size:0.85em; color:#94a3b8; border:1px solid #1e293b;'>
            From: <span style='color:#f0abfc;'>/content/download</span><br>
            To  : <span style='color:#f0abfc;'>{dest}</span>
        </div>
        """))

        ok_count = 0
        for fname in files:
            src_path  = os.path.join(SRC_DIR, fname)
            dest_path = os.path.join(dest, fname)
            try:
                shutil.copy2(src_path, dest_path)
                size = format_size(os.path.getsize(dest_path))
                display(HTML(f"""
                <div style='font-family:monospace; font-size:0.82em; color:#22c55e; margin:2px 0;'>
                    [OK] {fname} &nbsp;<span style='color:#475569;'>({size})</span>
                </div>
                """))
                ok_count += 1
            except Exception as e:
                display(HTML(f"""
                <div style='font-family:monospace; font-size:0.82em; color:#f87171; margin:2px 0;'>
                    [!!] {fname} — {e}
                </div>
                """))

        display(HTML(f"""
        <div style='background:#14532d; border-left:4px solid #22c55e; border-radius:6px;
             padding:12px 16px; margin-top:10px; font-family:sans-serif; color:#bbf7d0;'>
            <strong>Transfer complete.</strong> {ok_count}/{len(files)} file(s) moved to
            <code>{dest}</code>.
        </div>
        """))

transfer_btn.on_click(on_transfer)
refresh_btn.on_click(on_refresh)

display(HTML('<div style="font-family:monospace; font-size:0.75em; color:#475569; letter-spacing:2px; margin-bottom:8px;">TRANSFER TO GOOGLE DRIVE</div>'))
display(drive_path)
display(widgets.HBox([transfer_btn, refresh_btn], layout=widgets.Layout(margin='12px 0')))
display(transfer_out)

# Show current files on load
with transfer_out:
    display(HTML(list_files_html()))
